In [1]:
import cv2 ,os ,json
import albumentations as A
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image 

In [2]:
with open('./via_project.json') as file:
  db = json.load(file)
filter = ['filename','x','y','width','height','cat','dog']
db

{'1.jpeg45602': {'filename': '1.jpeg',
  'size': 45602,
  'regions': [{'shape_attributes': {'name': 'rect',
     'x': 323,
     'y': 36,
     'width': 320,
     'height': 479},
    'region_attributes': {'name': 'dog',
     'type': 'unknown',
     'image_quality': {'good': True,
      'frontal': True,
      'good_illumination': True}}}],
  'file_attributes': {'caption': '', 'public_domain': 'no', 'image_url': ''}},
 '2.jpeg4669': {'filename': '2.jpeg',
  'size': 4669,
  'regions': [{'shape_attributes': {'name': 'rect',
     'x': 0,
     'y': 47,
     'width': 160,
     'height': 112},
    'region_attributes': {'name': 'dog',
     'type': 'unknown',
     'image_quality': {'good': True,
      'frontal': True,
      'good_illumination': True}}},
   {'shape_attributes': {'name': 'rect',
     'x': 163,
     'y': 68,
     'width': 97,
     'height': 80},
    'region_attributes': {'name': 'cat',
     'type': 'unknown',
     'image_quality': {'good': True,
      'frontal': True,
      'good_ill

In [3]:
def get_depth(dic):
    list_ = []
    
    def val(key, value):
        if key in filter or value in filter:
            list_.append(str(f'{key}:{value}'))

    def check_dic(dic):
        if isinstance(dic,dict):
            for k,v in dic.items():
                if isinstance(v, (dict, list)):
                    check_dic(v)
                else:
                    val(k,v)
        elif isinstance(dic , (list)):
            for d in dic:
                if isinstance(d, (dict, list)):
                    check_dic(d)
                # else:                
                #     val(k,v)

    check_dic(dic)
    return list_
main_data = get_depth(db)
main_data

['filename:1.jpeg',
 'x:323',
 'y:36',
 'width:320',
 'height:479',
 'name:dog',
 'filename:2.jpeg',
 'x:0',
 'y:47',
 'width:160',
 'height:112',
 'name:dog',
 'x:163',
 'y:68',
 'width:97',
 'height:80',
 'name:cat',
 'filename:3.jpeg',
 'x:34',
 'y:1',
 'width:191',
 'height:151',
 'name:cat',
 'filename:4.jpeg',
 'x:67',
 'y:379',
 'width:823',
 'height:1356',
 'name:cat',
 'filename:5.jpeg',
 'x:12',
 'y:86',
 'width:770',
 'height:705',
 'name:dog',
 'filename:6.jpeg',
 'x:80',
 'y:48',
 'width:168',
 'height:226',
 'name:dog',
 'x:252',
 'y:137',
 'width:235',
 'height:142',
 'name:cat',
 'filename:7.jpeg',
 'x:22',
 'y:245',
 'width:608',
 'height:212',
 'name:cat',
 'x:586',
 'y:169',
 'width:301',
 'height:277',
 'name:dog']

In [4]:
if not os.path.exists('aug_images'):
    os.mkdir('aug_images')


def albumentation(name, box):
    image = cv2.imread(f"./images/{name}")
    cv2.imwrite(f"./aug_images/{name}",image)
    bboxes = box
    transform = A.Compose(
        [
            A.Resize(width=1920, height=1080),
            # A.RandomCrop(width=1280, height = 720),
            A.Rotate(limit=40, p=0.9, border_mode=cv2.BORDER_CONSTANT),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.1),
            A.RGBShift(r_shift_limit=25, g_shift_limit=25,
                       b_shift_limit=25, p=0.9),
            A.OneOf([
                A.Blur(blur_limit=3, p=0.5),
                A.ColorJitter(p=0.5),
            ], p=1.0),
        ], bbox_params=A.BboxParams(format="pascal_voc", label_fields=[])
    )

    # images_list = [image]
    # saved_bboxes = [[bboxes[0]]]
    for i in range(2):
        augmentations = transform(image=image, bboxes=bboxes)
        augmented_img = augmentations["image"]
        # images_list.append(augmented_img)
        # saved_bboxes.append(augmentations["bboxes"][0])
        img = augmented_img
        # print(augmentations["bboxes"][0])
        for j in range(len(augmentations["bboxes"])):
            x, y, w, h, cls = map(int, augmentations["bboxes"][j])
            f_name=f"aug{i}_{name}"
            cv2.imwrite(os.path.join('aug_images',f_name), img)
            f_size=os.path.getsize(os.path.join('aug_images',f_name))
            fname_size_key=f_name+str(f_size)
            if cls==0:
                cls='dog'
            else:
                cls='cat'
            if fname_size_key in db:
                db[fname_size_key]['regions'].append(
                    {'shape_attributes': {'name': 'rect',
                                                                'x': x,
                                                                'y': y,
                                                                'width': w-x,
                                                                'height': h-y},
                                            'region_attributes': {'name': cls,
                                                                'type': 'unknown',
                                                                'image_quality': {'good': True,
                                                                                    'frontal': True,
                                                                                    'good_illumination': True}}}
                )
            else:
                db[fname_size_key]= {'filename': f_name,
                                'size': f_size,
                                'regions': [{'shape_attributes': {'name': 'rect',
                                                                'x': x,
                                                                'y': y,
                                                                'width': w-x,
                                                                'height': h-y},
                                            'region_attributes': {'name': cls,
                                                                'type': 'unknown',
                                                                'image_quality': {'good': True,
                                                                                    'frontal': True,
                                                                                    'good_illumination': True}}}],
                                'file_attributes': {'caption': '', 'public_domain': 'no', 'image_url': ''}}


In [5]:
def box (main_data):
    list_ = []
    list_s = []
    data_dic={}
    xx = ['x','y', 'width', 'height']
    for i in main_data:
        if i.split(":")[0] == 'filename':
            if list_:
                data_dic[file_name]=list_
            list_=[]
            file_name=i.split(':')[1]
        elif i.split(":")[0] in xx:
                list_s.append(i.split(':')[1])
        if i.split(":")[0] == 'name':
                if i.split(":")[1]=='dog':
                    cls=0
                elif i.split(":")[1]=='cat':
                    cls=1
                x,y,w,h=map(int,list_s)
                list_s=[x,y,x+w,y+h,cls]
                list_.append(list_s) 
                list_s =[]
    if list_:
                data_dic[file_name]=list_
    return data_dic                

In [6]:
da=box(main_data)
for i in da:
    albumentation(i,da[i])
new_data=json.dumps(db,indent=4)
with open('new_json.json','w') as f:
    f.write(new_data)